In [1]:
import numpy as np
import pandas as pd
import scipy.io as sio
import matplotlib
import matplotlib.pyplot as plt
import random

from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score

Load in the Data

In [ ]:
games = pd.read_csv("C:/Users/jesse/Desktop/CSE141L/COGS118BProject/Data/nfl-big-data-bowl-2024/games.csv")
plays = pd.read_csv("C:/Users/jesse/Desktop/CSE141L/COGS118BProject/Data/nfl-big-data-bowl-2024/plays.csv")
plays

,gameId,playId,ballCarrierId,ballCarrierDisplayName,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,gameClock,preSnapHomeScore,preSnapVisitorScore,passResult,passLength,penaltyYards,prePenaltyPlayResult,playResult,playNullifiedByPenalty,absoluteYardlineNumber,offenseFormation,defendersInTheBox,passProbability,preSnapHomeTeamWinProbability,preSnapVisitorTeamWinProbability,homeTeamWinProbabilityAdded,visitorTeamWinProbilityAdded,expectedPoints,expectedPointsAdded,foulName1,foulName2,foulNFLId1,foulNFLId2
0,2022100908,3537,48723,Parker Hesse,(7:52) (Shotgun) M.Mariota pass short middle t...,4,1,10,ATL,TB,ATL,41,7:52,21,7,C,6.0,NaN,9,9,N,69,SHOTGUN,7.0,0.747284,0.976785,0.023215,-0.006110,0.006110,2.360609,0.981955,NaN,NaN,NaN,NaN
1,2022091103,3126,52457,Chase Claypool,(7:38) (Shotgun) C.Claypool right end to PIT 3...,4,1,10,PIT,CIN,PIT,34,7:38,14,20,NaN,NaN,NaN,3,3,N,76,SHOTGUN,7.0,0.416454,0.160485,0.839515,-0.010865,0.010865,1.733344,-0.263424,NaN,NaN,NaN,NaN
2,2022091111,1148,42547,Darren Waller,(8:57) D.Carr pass short middle to D.Waller to...,2,2,5,LV,LAC,LV,30,8:57,10,3,C,11.0,NaN,15,15,N,40,I_FORM,6.0,0.267933,0.756661,0.243339,-0.037409,0.037409,1.312855,1.133666,NaN,NaN,NaN,NaN
3,2022100212,2007,46461,Mike Boone,(13:12) M.Boone left tackle to DEN 44 for 7 ya...,3,2,10,DEN,LV,DEN,37,13:12,19,16,NaN,NaN,NaN,7,7,N,47,SINGLEBACK,6.0,0.592704,0.620552,0.379448,-0.002451,0.002451,1.641006,-0.043580,NaN,NaN,NaN,NaN
4,2022091900,1372,47857,Devin Singletary,(8:33) D.Singletary right guard to TEN 32 for ...,2,1,10,BUF,TEN,TEN,35,8:33,7,7,NaN,NaN,NaN,3,3,N,75,I_FORM,7.0,0.470508,0.836290,0.163710,0.001053,-0.001053,3.686428,-0.167903,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12481,2022100204,123,43293,Ezekiel Elliott,(13:31) E.Elliott right tackle to WAS 38 for 1...,1,1,10,DAL,WAS,WAS,39,13:31,0,0,NaN,NaN,NaN,1,1,N,49,SINGLEBACK,6.0,0.577800,0.638600,0.361400,-0.011542,0.011542,3.642571,-0.504018,NaN,NaN,NaN,NaN
12482,2022091200,3467,46189,Will Dissly,(6:08) G.Smith pass short right to W.Dissly to...,4,1,10,SEA,DEN,SEA,30,6:08,17,16,C,0.0,NaN,2,2,N,40,SINGLEBACK,7.0,0.298983,0.615241,0.384759,-0.025458,0.025458,1.434580,-0.444642,NaN,NaN,NaN,NaN
12483,2022101605,3371,44860,Joe Mixon,(9:35) (Shotgun) J.Mixon left end to CIN 47 fo...,4,1,10,CIN,NO,CIN,41,9:35,26,21,NaN,NaN,NaN,6,6,N,69,SHOTGUN,6.0,0.639439,0.667054,0.332946,-0.005164,0.005164,2.115356,0.203819,NaN,NaN,NaN,NaN
12484,2022100207,2777,52449,Jonathan Taylor,(2:02) (Shotgun) J.Taylor up the middle to TEN...,3,1,10,IND,TEN,TEN,34,2:02,17,24,NaN,NaN,NaN,-2,-2,N,44,SHOTGUN,6.0,0.518695,0.410611,0.589389,-0.046648,0.046648,3.946232,-0.976039,NaN,NaN,NaN,NaN


In [ ]:
plays = pd.read_csv("C:/Users/jesse/Desktop/CSE141L/COGS118BProject/Data/nfl-big-data-bowl-2024/plays.csv")
#makes pass result binary(1 run, 0 pass)
plays["run"] = plays["passResult"].isna()
plays_filtered = plays[["gameId","quarter", "down", "yardsToGo", "possessionTeam","defensiveTeam","absoluteYardlineNumber", "gameClock", "preSnapHomeScore","preSnapVisitorScore", "run","offenseFormation", "defendersInTheBox","expectedPoints"]]

games_filtered = games[["gameId", "week","gameTimeEastern"]]
#combine game data with play data
data = plays_filtered.merge(games_filtered, on="gameId")

#make all game times purly numbers (probably not necessary - most models can handle datetime or can convert to int differently)
data['gameClock'] = data['gameClock'].replace(':', '', regex =True).astype(int)
data['gameTimeEastern'] = data['gameTimeEastern'].replace(':', '', regex =True).astype(int)
data.drop("gameId",axis =1, inplace=True)

#1 hot encode teams and formations
teams = data.possessionTeam.unique()
data = pd.get_dummies(data, prefix=['possessionTeam', 'defensiveTeam', 'offenseFormation'])

Seperating plays by team and creating a test and train dataset for each team(later we can do analysis on which teams we can predict the best)

In [48]:

#empty dictionary of team names
plays_by_team_test = {x : pd.DataFrame() for x in teams}
plays_by_team_train = {x : pd.DataFrame() for x in teams}
plays_test = pd.DataFrame()
plays_train = pd.DataFrame()
#fills dictionary with all plays according to possesion team
for team in teams:
    plays_by_team_test[team] = data[data["possessionTeam_"+str(team)]]
    #seperate into train and test sets seperated by team 
    rows = len(plays_by_team_test[team])
    rand_idx = np.random.randint(0, rows,size = int(rows/10))
    
    
    #make seperate traina nd test sets for each team
    plays_by_team_train[team] = plays_by_team_test[team].drop(plays_by_team_test[team].index[rand_idx])
    plays_by_team_test[team] = plays_by_team_test[team].iloc[rand_idx]
    #create a joined train and test set(for overall)
    plays_test = pd.concat([plays_test, plays_by_team_test[team]], ignore_index = True)
    plays_train = pd.concat([plays_train, plays_by_team_train[team]], ignore_index = True)

#Randomize final sets to mix teams
plays_test = plays_test.sample(frac=1)
plays_train = plays_test.sample(frac=1)
#split datasets into parameters and result
plays_test_run = plays_test["run"]
plays_test_epa = plays_test["expectedPoints"]
plays_test.drop(["run","expectedPoints"],axis =1, inplace=True)
plays_train_run = plays_train["run"]
plays_train_epa = plays_train["expectedPoints"]
plays_train.drop(["run","expectedPoints"],axis =1, inplace=True)

#print to show
print("Pass Results")
print(plays_test_run)
print("Test Data")
plays_test

Pass Results
234     False
222      True
497     False
710      True
472     False
        ...  
103     False
499      True
172     False
360     False
1225    False
Name: run, Length: 1233, dtype: bool
Test Data


,quarter,down,yardsToGo,absoluteYardlineNumber,gameClock,preSnapHomeScore,preSnapVisitorScore,defendersInTheBox,week,gameTimeEastern,possessionTeam_ARI,possessionTeam_ATL,possessionTeam_BAL,possessionTeam_BUF,possessionTeam_CAR,possessionTeam_CHI,possessionTeam_CIN,possessionTeam_CLE,possessionTeam_DAL,possessionTeam_DEN,possessionTeam_DET,possessionTeam_GB,possessionTeam_HOU,possessionTeam_IND,possessionTeam_JAX,possessionTeam_KC,possessionTeam_LA,possessionTeam_LAC,possessionTeam_LV,possessionTeam_MIA,possessionTeam_MIN,possessionTeam_NE,possessionTeam_NO,possessionTeam_NYG,possessionTeam_NYJ,possessionTeam_PHI,possessionTeam_PIT,possessionTeam_SEA,possessionTeam_SF,possessionTeam_TB,possessionTeam_TEN,possessionTeam_WAS,defensiveTeam_ARI,defensiveTeam_ATL,defensiveTeam_BAL,defensiveTeam_BUF,defensiveTeam_CAR,defensiveTeam_CHI,defensiveTeam_CIN,defensiveTeam_CLE,defensiveTeam_DAL,defensiveTeam_DEN,defensiveTeam_DET,defensiveTeam_GB,defensiveTeam_HOU,defensiveTeam_IND,defensiveTeam_JAX,defensiveTeam_KC,defensiveTeam_LA,defensiveTeam_LAC,defensiveTeam_LV,defensiveTeam_MIA,defensiveTeam_MIN,defensiveTeam_NE,defensiveTeam_NO,defensiveTeam_NYG,defensiveTeam_NYJ,defensiveTeam_PHI,defensiveTeam_PIT,defensiveTeam_SEA,defensiveTeam_SF,defensiveTeam_TB,defensiveTeam_TEN,defensiveTeam_WAS,offenseFormation_EMPTY,offenseFormation_I_FORM,offenseFormation_JUMBO,offenseFormation_PISTOL,offenseFormation_SHOTGUN,offenseFormation_SINGLEBACK,offenseFormation_WILDCAT
234,3,3,20,25,152,22,16,3.0,4,162500,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False
222,2,1,7,17,1250,0,17,7.0,7,162500,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False
497,2,1,10,32,23,6,3,6.0,6,160500,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False
710,2,1,10,70,848,0,10,6.0,1,202000,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
472,3,3,20,48,422,16,21,4.0,7,162500,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [49]:
plays_test_run = np.ravel(np.array(plays_test_run))
plays_test_epa = np.ravel(np.array(plays_test_epa))
plays_test_param = np.array(plays_test)
plays_train_run = np.ravel(np.array(plays_train_run))
plays_train_epa = np.ravel(np.array(plays_train_epa))
plays_train_param = np.array(plays_train)

Testing Out Sklearn Logistic Regression to see how good the Data is

In [ ]:
model = LinearRegression()
model.fit(plays_train_param, plays_train_run)
pass_pred = model.predict(plays_test_param)
pass_pred = np.where(pass_pred>.5,1,0)
matching = np.sum(pass_pred == plays_test_run)
#percentage accuarcy
matching/len(pass_pred) * 100

76.23682076236821

In [ ]:
tackles_df = pd.read_csv("C:/Users/jesse/Desktop/CSE141L/COGS118BProject/Data/nfl-big-data-bowl-2024/tackles.csv")
tackles_df

,gameId,playId,nflId,tackle,assist,forcedFumble,pff_missedTackle
0,2022090800,101,42816,1,0,0,0
1,2022090800,393,46232,1,0,0,0
2,2022090800,486,40166,1,0,0,0
3,2022090800,646,47939,1,0,0,0
4,2022090800,818,40107,1,0,0,0
...,...,...,...,...,...,...,...
17421,2022091113,2494,43533,0,0,0,1
17422,2022092502,3510,42406,0,0,0,1
17423,2022091113,3642,43478,0,0,0,1
17424,2022091901,3578,42431,0,0,0,1
